In [ ]:
! pip install imutils

In [4]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import os

In [5]:
import keras
import keras.utils
from keras import utils as np_utils
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
import random
import plotly.graph_objects as go

# Image Detection

In [ ]:
data_path='/Users/mnoor/Desktop/FaceMaskProject/FaceMask/img'
CATEGORIES = ['with_mask','without_mask']

In [ ]:
training_data = []
IMG_SIZE=128
def making_trian_dataset():
    for category in CATEGORIES:
        path = os.path.join(data_path, category) # path to our data
        class_num = CATEGORIES.index(category)# classifcation index
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) # resizing our imgz
                training_data.append([new_array, class_num])
            except Exception as e:
                pass
making_trian_dataset()

In [ ]:
print(len(training_data))

In [ ]:
random.shuffle(training_data)
train_data, labels_data = zip(*training_data)

In [ ]:
for Sample in training_data[:10]:
    print(Sample[1])

In [ ]:
X=[]
y=[]
for features, label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X)#.reshape(-1, IMG_SIZE,IMG_SIZE,3)
y = to_categorical(y, num_classes = 3)

In [ ]:
X.shape,y.shape

Data Splitting For Simple NN

In [ ]:
nsample,nx,ny,ni=X.shape
X1=X.reshape(nsample,nx*ny*ni)
print(X1.shape)

In [ ]:
X1_train, X1_val_test, y1_train, y1_val_test = train_test_split(X1, y, test_size=.2, random_state=77)
X1_val, X1_test, y1_val, y1_test = train_test_split(X1_val_test, y1_val_test, test_size=0.5, random_state=77)
print(f"\nTraining data: {X1_train.shape},  labels: {y1_train.shape}")
print(f"Validation data: {X1_val.shape},  labels: {y1_val.shape}")
print(f"Testing data: {X1_test.shape},  labels: {y1_test.shape}")

Data Splitting

In [ ]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=.2, random_state=77)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=77)
print(f"\nTraining data: {X_train.shape},  labels: {y_train.shape}")
print(f"Validation data: {X_val.shape},  labels: {y_val.shape}")
print(f"Testing data: {X_test.shape},  labels: {y_test.shape}")

Visualizing our data

In [ ]:
Mask_path='/Users/mnoor/Desktop/FaceMaskProject/FaceMask/img/with_mask'
No_mask='/Users/mnoor/Desktop/FaceMaskProject/FaceMask/img/without_mask'

In [ ]:
fig = go.Figure(
    data=[go.Pie(labels=['with_mask','without_mask'], 
        values=[len(os.listdir(Mask_path)),len(os.listdir(No_mask))])
    ])
fig.show()

Distribution of the target variables

In [ ]:
def display_random_set(data, labels, classes):
    plt.figure(figsize=(15, 15))
    for i in range(10):
        plt.subplot(5, 5, (i+1))
        random_val = np.random.randint(low=0, high=len(data))
        img = data[random_val]
        plt.imshow(img)
        plt.axis(False)
        plt.title(classes[np.argmax(labels[random_val])])
    plt.show()

In [ ]:
display_random_set(data=X_train, labels=y_train, classes=CATEGORIES)

Model Transfer Learning

In [ ]:
from tensorflow.keras.layers import Input
from keras.applications.vgg19 import VGG19  
from keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Dropout, Flatten,InputLayer
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import mobilenet_v2

MobileNetV2

In [ ]:
model_3 = mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, input_shape=(128,128,3)) 
 
for layer in model_3.layers:
    layer.trainable = False    

x = model_3.output
x = Flatten()(x) 
x = Dense(100, activation='relu')(x) 
x = Dense(50, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x) 

model_3 = Model(inputs=model_3.input, outputs=predictions)
model_3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_3.summary()

In [ ]:
model_3.fit(X_train, y_train, validation_data=(X_val,y_val),epochs=10)

VGG19

In [ ]:
model_6 = VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))  

# Freeze convolutional layers
for layer in model_6.layers:
    layer.trainable = False  

NN_transfer_19 = Sequential(
                        [InputLayer(input_shape=(128,128,3)),model_6,
                         Flatten(),  # should be fine , or add layers
                         Dense(128, activation='relu'),
                         Dense(64, activation='relu'),
                         Dense(32, activation='relu'),   # 2 dense is must bcuz VGG16 model Conv2D twice and Maxpooling -> get a lot more features
                         Dense(3, activation='softmax')]
                       )

NN_transfer_19.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'],)
model_6.summary()

In [ ]:
result_V19 = NN_transfer_19.fit(X_train,y_train, validation_data=(X_val,y_val), epochs=10)

In [ ]:
acc = result_V19.history['accuracy']
val_acc = result_V19.history['val_accuracy']

loss = result_V19.history['loss']
val_loss = result_V19.history['val_loss']

epochs=10
epochs_range = range(epochs)


plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Training Accuracy', color = 'red') #, color = 'red')
plt.plot(epochs_range, val_acc, label='Validation Accuracy',color='blue')  # , color='blue')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')


plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss',color='red')
plt.plot(epochs_range, val_loss, label='Validation Loss',color='blue')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

In [ ]:
print("[INFO] saving mask detector model...")
model_6.save("Face-Mask-Detection-model_6.model", save_format="h5")

# Video detection

In [ ]:
import cv2

In [ ]:
# This is the config file 
config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'

# This is from the weight file 
frozen_model = 'frozen_inference_graph.pb'

In [ ]:
# Create an instance from model detection

model = cv2.dnn_DetectionModel(frozen_model, config_file)

In [ ]:
class_labels = []
file_name = 'labels.txt'
with open(file_name, 'rt') as fpt:
    class_labels = fpt.read().rstrip('\n').split('\n')

In [ ]:
print(class_labels)

In [ ]:
len(class_labels)

In [ ]:
model.setInputSize(320,320)
# Note: 255/2 = 127.5
model.setInputScale(1.0/127.5)

model.setInputMean((127.5,127.5,127.5))

model.setInputSwapRB(True)

In [ ]:
# Record any vidoe and add it here for a demo to the students 
import cv2
#cap = cv2.VideoCapture('videos/car_traffic.mp4')
cap = cv2.VideoCapture('/Users/mnoor/Desktop/FaceMaskProject/Video/Video1.mp4')

# Check if video is open correclty 
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
    
if not cap.isOpened():
    raise IOError('Cannot Open Video File')
    
# Text font
font = cv2.FONT_HERSHEY_SIMPLEX
# Font scale
fontScale = 2 
# Box color
box_color = (255, 0, 0)
# Teaxt color
text_color = (0, 255, 0)
# Line thickness of 2 px
thickness = 2
   
while True:
    ret,frame = cap.read()
    
    index, confidence, bbox = model.detect(frame)
    
    print(index)
    if(len(index) !=0):
        for index, conf, box in zip(index.flatten(), confidence.flatten(), bbox):
            if(index <=2):
                cv2.rectangle(frame, box, (255,0,0), 2)
                cv2.putText(frame, class_labels[index-1],(box[0]+10, box[1]+40), font, fontScale = fontScale, color = text_color, thickness = thickness)

    cv2.imshow('Object Detection Demo', frame)
    
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


# LIVE Detection

In [7]:
from tensorflow.keras.models import Sequential

ModuleNotFoundError: No module named 'tensorflow.keras'

In [6]:
model =Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [ ]:
TRAINING_DIR = "/Users/mnoor/Desktop/FaceMaskProject/FaceMask/train"

train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))

In [ ]:
VALIDATION_DIR = "/Users/mnoor/Desktop/FaceMaskProject/FaceMask/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')


history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

In [2]:
import cv2
import numpy as np
from keras.models import load_model
#model=load_model("model2-010.model")

labels_dict={0:'without mask',1:'mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

Using TensorFlow backend.
2022-11-05 20:56:42.550670: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


IsADirectoryError: [Errno 21] Unable to open file (file read failed: time = Sat Nov  5 20:56:54 2022
, filename = 'model2-010.model', file descriptor = 77, errno = 21, error message = 'Is a directory', buf = 0x7ffee36453f8, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)

In [ ]:
# We load the xml file
classifier = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

In [ ]:
while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1)
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))
    faces = classifier.detectMultiScale(mini)
    for f in faces: 
        (x, y, w, h) = [v * size for v in f]
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1) 
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',im)
    key = cv2.waitKey(10)
    if key == 27: #The Esc key 
        break
webcam.release()
cv2.destroyAllWindows()